In [1]:
# Cell 1: imports & basic config

import os
import json
import requests
import pandas as pd

pd.set_option("display.max_columns", 50)

In [5]:
# Cell 2: paths and constants

# 如果 business.json 就在当前 notebook 同一文件夹，用 "."
DATA_DIR = "."
BUSINESS_PATH = os.path.join(DATA_DIR, "yelp_academic_dataset_business.json")

# Census API settings
YEAR = 2023
BASE_URL = f"https://api.census.gov/data/{YEAR}/acs/acs5/profile"

# TODO: 把这里替换成你自己的 Census API key
CENSUS_API_KEY = "YOUR_CENSUS_API_KEY_HERE"

# Variables we want from ACS Data Profiles
ACS_VARS = [
    "NAME",
    "DP03_0062E",   # median household income (dollars)
    "DP02_0068PE",  # % 25+ with bachelor's degree or higher
    "DP05_0001E",   # total population
]


In [22]:
# Cell 3 (new): collect all Arizona ZIP codes from Yelp business.json

phoenix_zips = set()  # 名字先不改，后面代码还能直接用

with open(BUSINESS_PATH, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        # 只要亚利桑那州的业务
        if obj.get("state") != "AZ":
            continue

        z = obj.get("postal_code")
        if not z:
            continue

        z_str = str(z).strip()
        if not z_str:
            continue

        # 统一成 5 位 ZIP
        phoenix_zips.add(z_str.zfill(5))

print(f"Found {len(phoenix_zips)} unique AZ ZIP codes in Yelp business data.")
print(sorted(list(phoenix_zips))[:20], "...")


Found 62 unique AZ ZIP codes in Yelp business data.
['32246', '85226', '85271', '85614', '85619', '85623', '85629', '85641', '85652', '85653', '85654', '85658', '85701', '85702', '85703', '85704', '85705', '85706', '85707', '85708'] ...


In [24]:
# Cell 4: (optional) save Phoenix ZIP list

phoenix_zips_df = pd.DataFrame(sorted(list(phoenix_zips)), columns=["zip"])
phoenix_zips_df.to_csv("phoenix_zips_from_yelp.csv", index=False)
phoenix_zips_df.head()


,zip
0,32246
1,85226
2,85271
3,85614
4,85619


In [26]:
# Cell 5: call ACS API for all ZCTAs

params = {
    "get": ",".join(ACS_VARS),
    "for": "zip code tabulation area:*",
}

if CENSUS_API_KEY and CENSUS_API_KEY != "YOUR_CENSUS_API_KEY_HERE":
    params["key"] = CENSUS_API_KEY

response = requests.get(BASE_URL, params=params)
response.raise_for_status()

data = response.json()
print("Number of rows (including header):", len(data))


Number of rows (including header): 33773


In [27]:
# Cell 6: build DataFrame & basic cleaning

header = data[0]
rows = data[1:]

df_acs_all = pd.DataFrame(rows, columns=header)

# rename ZIP column
df_acs_all = df_acs_all.rename(columns={"zip code tabulation area": "zip"})

# ensure zip is 5-char string
df_acs_all["zip"] = df_acs_all["zip"].astype(str).str.zfill(5)

# convert numeric columns
numeric_cols = [c for c in df_acs_all.columns if c not in ["NAME", "zip", "state"]]
for col in numeric_cols:
    df_acs_all[col] = pd.to_numeric(df_acs_all[col], errors="coerce")

df_acs_all.head()


,NAME,DP03_0062E,DP02_0068PE,DP05_0001E,zip
0,ZCTA5 00601,18571,NaN,16721,00601
1,ZCTA5 00602,21702,NaN,37510,00602
2,ZCTA5 00603,19243,NaN,48317,00603
3,ZCTA5 00606,20226,NaN,5435,00606
4,ZCTA5 00610,23732,NaN,25413,00610


In [28]:
# Cell 7: filter ACS data to Phoenix-metro ZIPs

df_acs_phx = df_acs_all[df_acs_all["zip"].isin(phoenix_zips)].copy()

print(df_acs_phx.shape)
df_acs_phx.head()


(47, 5)


,NAME,DP03_0062E,DP02_0068PE,DP05_0001E,zip
10592,ZCTA5 32246,78506,39.2,60443,32246
29710,ZCTA5 85226,103740,50.0,40974,85226
29841,ZCTA5 85614,62599,42.2,24696,85614
29846,ZCTA5 85619,-666666666,50.0,22,85619
29850,ZCTA5 85623,68219,40.6,5235,85623


In [29]:
# Cell 8: select / rename final columns and save to CSV

df_acs_phx_final = df_acs_phx[["zip", "DP03_0062E", "DP02_0068PE", "DP05_0001E"]].copy()

df_acs_phx_final = df_acs_phx_final.rename(columns={
    "DP03_0062E": "median_income",
    "DP02_0068PE": "pct_college",
    "DP05_0001E": "total_pop",
})

output_path = "acs_phx_2023.csv"
df_acs_phx_final.to_csv(output_path, index=False)

print(f"Saved ACS Phoenix ZIP-level data to: {output_path}")
df_acs_phx_final.head()


Saved ACS Phoenix ZIP-level data to: acs_phx_2023.csv


,zip,median_income,pct_college,total_pop
10592,32246,78506,39.2,60443
29710,85226,103740,50.0,40974
29841,85614,62599,42.2,24696
29846,85619,-666666666,50.0,22
29850,85623,68219,40.6,5235


In [30]:
# Cell 9: sanity checks

print(df_acs_phx_final.describe())

zips_from_acs = set(df_acs_phx_final["zip"])
missing_in_acs = phoenix_zips - zips_from_acs

print(f"\nZIPs in Yelp but not found in ACS: {len(missing_in_acs)}")
print(sorted(list(missing_in_acs))[:20])


       median_income   pct_college     total_pop
count   4.700000e+01  4.700000e+01     47.000000
mean   -1.275971e+08 -7.092195e+07  24261.808511
std     2.651822e+08  2.077737e+08  17578.138865
min    -6.666667e+08 -6.666667e+08      0.000000
25%     4.705100e+04  1.835000e+01   8064.000000
50%     6.821900e+04  3.720000e+01  24435.000000
75%     9.136250e+04  4.640000e+01  35662.000000
max     1.216470e+05  7.220000e+01  60443.000000

ZIPs in Yelp but not found in ACS: 15
['85271', '85652', '85702', '85703', '85717', '85728', '85731', '85732', '85733', '85740', '85751', '85752', '85754', '85812', '86712']


In [37]:
# 把 ACS 的特殊缺失值 -666666666 替换成 NaN
df_acs_phx_final = df_acs_phx_final.replace(-666666666, pd.NA)

# 可选：再看一眼描述统计
df_acs_phx_final.describe()


,total_pop
count,47.000000
mean,24261.808511
std,17578.138865
min,0.000000
25%,8064.000000
50%,24435.000000
75%,35662.000000
max,60443.000000


In [39]:
df_acs_phx_final.to_csv("acs_phx_2023.csv", index=False)
